Importing all neccessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns;sns.set(style="white")
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import random
from PIL import Image
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import keras
from keras.models import Sequential

from keras.layers import Convolution2D,MaxPooling2D,Dropout,Dense,Flatten,BatchNormalization,Conv2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import preprocessing
from keras.utils import np_utils


import warnings
warnings.simplefilter("ignore")

Loading Dataset

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = ''  # Your Kaggle username
os.environ['KAGGLE_KEY'] = ''  # Your Kaggle API key
os.environ['URN'] = ''  # Your URN: submissions without a URN will not count

In [ ]:
!python3 /usr/local/bin/kaggle -v
!python3 -m pip show kaggle

In [ ]:
!python3 /usr/local/bin/kaggle competitions download -c uos-com2028-21-22-cw
!unzip uos-com2028-21-22-cw.zip

In [ ]:
df=pd.read_csv("train.csv")
df["ID"] = ["0"* (6-len(str(i))) + str(i) if len(str(i)) < 6 else i for i in df["Id"]]
df.head()

In [ ]:
df["Cell type"].value_counts()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize= (20, 5))
sns.countplot(x = 'Cell type' , data = df, order = df['Cell type'].value_counts().index, palette="tab10")

Preprocessing

In [ ]:
data=[]
labels=[] 

for i in os.listdir("train/"):
  try:
    image = cv2.imread("train/"+i)
    image_array = Image.fromarray(image , 'RGB')
    resize_img = image_array.resize((50 , 50))
    dx = df.loc[df["ID"]==i.split(".")[0], "Cell type"].values[0]
    data.append(np.array(resize_img))
    labels.append(dx)
        
  except:
    pass


data = np.array(data)
labels = np.array(labels)

print('Data : {} | labels : {}'.format(data.shape , len(labels)))

In [ ]:
np.save('data' , data)
np.save('labels' , labels)

In [ ]:
uni_labels = df["Cell type"].unique().tolist()

for uni in uni_labels:
  ls=[]
  for label in range(len(labels)):
    if labels[label] == uni:
      ls.append(label)
  ls =  random.choices(population=ls, k=5)

  plt.figure(figsize=(15,5))
  for i in range(1,6):
    plt.subplot(1,5,i)
    plt.imshow(data[ls[i-1]])
    plt.title("Class: "+str(uni))
    plt.xticks([])
    plt.yticks([])

Train Test Split

In [ ]:
train_x , test_x , train_y , test_y = train_test_split(data , labels , test_size = 0.3, stratify = labels)

In [ ]:
plt.figure(1 , figsize = (20 ,5))
n = 0 
for z , j in zip([train_y , test_y] , ['train labels', 'test labels']):
  n += 1
  plt.subplot(1 , 3  , n)
  sns.countplot(x = z )
  plt.title(j)
plt.show()


train_y=np_utils.to_categorical(train_y,num_classes=8)
test_y=np_utils.to_categorical(test_y,num_classes=8)

print("Y train shape",train_y.shape)

CNN

In [ ]:
keras.backend.clear_session()

cnn = Sequential()
cnn.add(Convolution2D(32,(3,3),activation='relu',input_shape = (50, 50, 3)))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(2,2))
cnn.add(Dropout(0.2))
cnn.add(Convolution2D(32,(3,3),activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(2,2))
cnn.add(Dropout(0.2))
cnn.add(Convolution2D(64,(3,3),activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(2,2))
cnn.add(Dropout(0.2))
cnn.add(Flatten())
cnn.add(Dense(64,activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(Dense(8,activation='softmax'))
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
cnn.summary()

In [ ]:
%%time
early_stopping_monitor = EarlyStopping(patience = 3) 
nn = cnn.fit(train_x, train_y, epochs = 10, validation_split = 0.2, callbacks = [early_stopping_monitor])

In [ ]:
values  = nn.history
validation_loss = values['val_loss']
validation_acc = values['val_accuracy']
training_acc = values['accuracy']
training_loss = values['loss']
epochs = range(1, len(training_acc) + 1)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 5)
plt.plot(epochs,training_loss,label = 'Training Loss')
plt.plot(epochs,validation_loss,label = 'Validation Loss')
plt.title('Epochs vs Loss')
plt.xlabel('Epochs')
plt.xticks(epochs)
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 5)
plt.plot(epochs,training_acc,label = 'Training Accuracy')
plt.plot(epochs,validation_acc,label = 'Validation Accuracy')
plt.title('Epochs vs Accuracy')
plt.xlabel('Epochs')
plt.xticks(epochs)
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
predictions= cnn.predict(test_x).round(0)
loss1, test_acc = cnn.evaluate(test_x, test_y)
test_acc = test_acc*100

loss2, train_acc = cnn.evaluate(train_x, train_y)
train_acc = train_acc*100

print('Test Loss score:', loss1)
print('Test accuracy:', test_acc)

print('Train Loss score:', loss2)
print('Train accuracy:', train_acc)

target = [0,1,2,3,4,5,6,7]
print("Classification Report: \n",classification_report(test_y, predictions)) 
print()
print("Confusion Matrix:")
cm = confusion_matrix(np.argmax(test_y, axis=1), np.argmax(predictions, axis=1))
cm =pd.DataFrame(cm, index=target, columns=target)
cm

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score
ypred = cnn.predict(test_x)
ypred = ypred.argmax(axis=-1)

# set plot figure size
fig, c_ax = plt.subplots( )
fig.set_size_inches(20, 8)
plt.xlim((0,1))
plt.ylim((0,1))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(test_y)
    y_test = lb.transform(test_y)
    y_pred = lb.transform(predictions)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
   
    return roc_auc_score(test_y, y_pred, average=average)


print('ROC AUC score:', multiclass_roc_auc_score(train_y, predictions))
plt.grid()
plt.plot([0, 1], [0, 1], 'k--')
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.show()
# visualizer.show() 

Resnet50

In [ ]:
from tensorflow.keras.applications import ResNet50

resnet = ResNet50(weights="imagenet",include_top = False,input_shape=(50,50,3))

for layer in resnet.layers:
  layer.trainable = False

In [ ]:
resnet50 = Sequential()
resnet50.add(resnet)
resnet50.add(Flatten())
resnet50.add(Dense(8,activation="softmax"))
resnet50.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
%%time

checkpoint = ModelCheckpoint("resnet50.h5", monitor="val_accuracy", verbose=1, save_best_only=True, save_weights_only=False, period=1)

earlystop = EarlyStopping(monitor="val_accuracy",patience=5,verbose=1)

history = resnet50.fit(train_x,train_y,batch_size=32,epochs=10, validation_data=(test_x,test_y), verbose=1, callbacks=[checkpoint,earlystop])

In [ ]:
values  = history.history
validation_loss = values['val_loss']
validation_acc = values['val_accuracy']
training_acc = values['accuracy']
training_loss = values['loss']
epochs = range(1, len(training_acc) + 1)

In [ ]:
# visualization of overall model performance 

plt.figure(1 , figsize = (20 ,5))
plt.plot(epochs,training_loss,label = 'Training Loss')
plt.plot(epochs,validation_loss,label = 'Validation Loss')
plt.title('Epochs vs Loss')
plt.xlabel('Epochs')
plt.xticks(epochs)
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# visualization of overall model performance 

plt.figure(1 , figsize = (20 ,5))
plt.plot(epochs,training_acc,label = 'Training Accuracy')
plt.plot(epochs,validation_acc,label = 'Validation Accuracy')
plt.title('Epochs vs Accuracy')
plt.xlabel('Epochs')
plt.xticks(epochs)
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
predictions= resnet50.predict(test_x).round(0)
loss1, test_acc = resnet50.evaluate(test_x, test_y)
test_acc = test_acc*100

loss2, train_acc = resnet50.evaluate(train_x, train_y)
train_acc = train_acc*100

print('Test Loss score:', loss1)
print('Test accuracy:', test_acc)

print('Train Loss score:', loss2)
print('Train accuracy:', train_acc)

print("Classification Report: \n",classification_report(test_y, predictions)) 
print()
print("Confusion Matrix:")
cm = confusion_matrix(np.argmax(test_y, axis=1), np.argmax(predictions, axis=1))
cm =pd.DataFrame(cm, index=target, columns=target)
cm

In [ ]:
ypred = resnet50.predict(test_x)
ypred = ypred.argmax(axis=-1)

# set plot figure size
fig, c_ax = plt.subplots( )
fig.set_size_inches(20, 8)
plt.xlim((0,1))
plt.ylim((0,1))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(test_y)
    y_test = lb.transform(test_y)
    y_pred = lb.transform(predictions)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
   
    return roc_auc_score(test_y, y_pred, average=average)


print('ROC AUC score:', multiclass_roc_auc_score(train_y, predictions))
plt.grid()
plt.plot([0, 1], [0, 1], 'k--')
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.show()

Testing Prediction

In [ ]:
test_data=[]

for i in os.listdir("test/"):
  try:
    image = cv2.imread("test/"+i)
    image_array = Image.fromarray(image , 'RGB')
    resize_img = image_array.resize((50 , 50))
    test_data.append(np.array(resize_img))     
  except:
    pass

test_data = np.array(test_data)
predictions= cnn.predict(test_data).round(0)

In [ ]:
predictions= predictions.argmax(axis=1)

In [ ]:
print(len(predictions))
print(len(os.listdir("test/")))
print(test_data.shape)

In [ ]:
id_map = [filename[:5] for filename in os.listdir("test/")]
prediction_df = pd.DataFrame({"Id": id_map, "Cell type":predictions})
prediction_df.head()

In [ ]:
prediction_df["Cell type"].value_counts()

In [ ]:
prediction_df.to_csv("prediction.csv", index=False)

In [ ]:
!python3 /usr/local/bin/kaggle competitions submit -m $URN -c uos-com2028-21-22-cw -f prediction.csv